# Inteligencia Artificial en Ciberseguridad
Modelo de clasificación para identificación de ataques (Brute Force, Probing and Crypto Minner) dentro del trafico del red

In [ ]:
# Importar las bibliotecas necesarias para cargar datos, dividir conjuntos,
# construir el modelo de vecinos más cercanos y evaluar el desempeño.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix
import matplotlib.pyplot as plt


Cargamos los datos desde el archivo CSV utilizando pandas.


In [ ]:
data = pd.read_csv("/kaggle/input/allflowmeter-hikari2021/ALLFLOWMETER_HIKARI2021.csv")

Visualizamos la estructura del conjunto de datos

In [ ]:
data.head(1)
data.tail(2)

Separamos las características y las etiquetas del conjunto de datos.



In [ ]:
# Crear un nuevo DataFrame "data_X" eliminando la columna "label".
data_X = data.drop(["label"], axis=1)

# Crear una Serie "data_y" que contiene solo la columna "label".
data_y = data["label"]

Dividimos los datos en conjuntos de entrenamiento y prueba.
Utilizamos train_test_split para dividir las características y las etiquetas en conjuntos de entrenamiento y prueba.

Se reserva el 30% de los datos para el conjunto de prueba y se establece una semilla aleatoria (random_state) en 55 para reproducibilidad.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.3, random_state=55)


# ENTRENAMIENTO Y EVALUACION DEL MODELO CON KNN
Creamos y entrenamos el modelo de clasificación basado en k vecinos más cercanos (KNN).

In [ ]:
# Creamos una instancia del modelo KNeighborsClassifier con 3 vecinos.
knn_model = KNeighborsClassifier(n_neighbors=3)

# Entrenamos el modelo utilizando el conjunto de entrenamiento.
knn_model.fit(X_train, y_train)

Realizamos predicciones con el modelo KNeighborsClassifier y evaluamos su desempeño.

In [ ]:
# Realizamos predicciones en el conjunto de prueba utilizando el modelo KNN.
prediccion_knn = knn_model.predict(X_test)

# Definimos las etiquetas de clase para el informe de clasificación.
clases = ['Normal', 'Ataque']

# Imprimimos un informe de clasificación que muestra métricas de evaluación como precision, recall y f1-score.
print(classification_report(y_test, prediccion_knn, target_names=clases))


Generamos una matriz de confusión para evaluar las predicciones del modelo comparando las etiquetas reales (y_test) con las predicciones del modelo (prediccion_knn).

In [ ]:
confusion_matrix(y_test, prediccion_knn)

Calculamos la curva ROC y el área bajo la curva (AUC) para evaluar el desempeño del modelo.

In [ ]:
# Calcular la curva ROC comparando las etiquetas reales (y_test) con las predicciones del modelo (prediccion_knn).
false_positive, true_positive, _ = roc_curve(y_test, prediccion_knn)

# Calculamos el área bajo la curva (AUC) para evaluar el desempeño del modelo.
auc = roc_auc_score(y_test, prediccion_knn)
auc


Graficar la curva ROC utilizando los valores de falsos positivos y verdaderos positivos.

In [ ]:
plt.plot(false_positive, true_positive, label="AUC="+str(auc))
plt.title("Area bajo la curva")
plt.ylabel("Verdaderos Positivos")
plt.xlabel("Falsos Positivos")
plt.legend(loc=4)

# ENTRENAMIENTO Y PRUEBAS DE MODELO CON RANDOM FOREST
Crear y entrenar un modelo de clasificación utilizando el algoritmo de Bosques Aleatorios (Random Forest).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Crear una instancia del modelo RandomForestClassifier.
classifier = RandomForestClassifier()

# Entrenar el modelo utilizando el conjunto de entrenamiento.
classifier.fit(X_train, y_train)


Realizamos predicciones con el modelo de Bosques Aleatorios y evalua su desempeño.,

In [ ]:
# Realizamos predicciones en el conjunto de prueba utilizando el modelo de Bosques Aleatorios.
prediction_RF = classifier.predict(X_test)

# Imprimimos un informe de clasificación que muestra métricas de evaluación como precision, recall y f1-score.
print(classification_report(y_test, prediction_RF))


Creamos un nuevo conjunto de datos y separamos características y etiquetas.


In [ ]:
# Seleccionamos las columnas 'flow_duration', 'fwd_pkts_per_sec' y 'label' del conjunto de datos original.
data_gr6 = data[["flow_duration", "fwd_pkts_per_sec", "label"]]

# Creamos un nuevo conjunto de características "data_gr6X" excluyendo la columna 'label'.
data_gr6X = data_gr6.drop(['label'], axis=1)

# Creamos una Serie "data_gr6y" que contiene solo la columna 'label'.
data_gr6y = data_gr6['label']


Dividir el nuevo conjunto de datos en conjuntos de entrenamiento y prueba.

In [ ]:
# Utilizamos train_test_split para dividir las características y las etiquetas del nuevo conjunto de datos.
# Se reserva el 30% de los datos para el conjunto de prueba y se establece una semilla aleatoria (random_state) en 55 para reproducibilidad.
XX_train, XX_test, yy_train, yy_test = train_test_split(data_gr6X, data_gr6y, test_size=0.3, random_state=55)

# ENTRENAMIENTO Y PRUEBAS DE MODELO CON REGRESION LOGISTICA
Creamos y entrenamos un modelo de regresión logística.


In [ ]:
from sklearn.linear_model import LogisticRegression

# Crear una instancia del modelo LogisticRegression.
regresionlog = LogisticRegression()

# Entrenar el modelo utilizando el conjunto de entrenamiento del nuevo conjunto de datos.
regresionlog.fit(XX_train, yy_train)


Realizar predicciones con el modelo de regresión logística y evaluar su desempeño.

In [ ]:
# Realizamos predicciones en el conjunto de prueba utilizando el modelo de regresión logística.
prediccion_RLog = regresionlog.predict(XX_test)

# Definimos las etiquetas de clase para el informe de clasificación.
clases_RLog = ['Normal', 'Ataque']

# Imprimimos un informe de clasificación que muestra métricas de evaluación como precision, recall y f1-score.
print(classification_report(yy_test, prediccion_RLog, target_names=clases_RLog))
